# BOOT CAMP

## DESCRIÇÃO DO PROJETO
Utilização de ferramentas computacionais de datascience para analisar a participação das mulheres nas eleições no Brasil.

## DATASET
* http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

## ODS utilizada
5.5 garantir a participação plena e efetiva das mulheres e a igualdade de oportunidades para a liderança em todos os níveis de tomada de decisão na vida política, econômica e pública.

## HIPÓTESES
* Verificar se a proporção de homens e mulheres candidatos é a mesma proporção dos eleitos.
* Verificar o quanto as despesas nas campanha influencia no sucesso da candidatura.
* Verificar o quanto a correlação entre as variáveis cor, sexo, grau de instrução, estado civil, idade influencia no sucesso da candidatura.


In [638]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split

Linha para mostrar os número sem notação científica

In [142]:
pd.set_option("display.float_format", lambda x : "%.3f" % x)

Ler o arquivo das eleições de 2014

In [16]:
candidatos_2014 = pd.read_csv("data/consulta_cand_2014_BRASIL.csv", low_memory=False, sep=';', encoding='latin', parse_dates=["DT_GERACAO"])

# Linhas para categorizar as variáveis do DataSet.

In [269]:
from pandas.api.types import is_string_dtype

for n,c in candidatos_2014.items():
    if is_string_dtype(c): candidatos_2014[n] = c.astype('category').cat.as_ordered()

# Fase exploratória

In [18]:
candidatos_2014.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO
0,2018-09-20,09:50:32,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,...,BRANCA,276,SENADOR,36000000,4,NÃO ELEITO,N,S,292452014,9117120146160000
1,2018-09-20,09:50:32,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,...,PARDA,999,OUTROS,20000000,4,NÃO ELEITO,N,N,101272014,9721920146020000
2,2018-09-20,09:50:32,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,...,BRANCA,999,OUTROS,2000000,5,SUPLENTE,N,N,763742014,12808120146190000
3,2018-09-20,09:50:32,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,...,BRANCA,131,ADVOGADO,5000000,5,SUPLENTE,N,S,775172014,24871820146190000
4,2018-09-20,09:50:32,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,...,BRANCA,266,PROFESSOR DE ENSINO MÉDIO,1000000,5,SUPLENTE,N,N,762962014,30501220146190000


In [642]:
candidatos_2014.describe()

,ANO_ELEICAO,CD_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,CD_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,CD_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,...,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,CD_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,CD_COR_RACA,CD_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,NR_PROTOCOLO_CANDIDATURA
count,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,...,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000,26264.000
mean,2014.000,1.998,1.003,143.229,6.727,150359428201.095,24606.318,36105450006.144,10.566,3.504,...,47.171,52790415579.707,2.620,6.689,3.119,1.819,419.938,2492303.064,3.870,447028532.026
std,0.000,0.040,0.051,5.699,0.787,74201689518.568,21944.618,28891396921.780,3.294,3.751,...,11.430,66955454973.419,0.925,1.492,2.300,0.956,320.619,5239804.836,1.999,384477884.722
min,2014.000,1.000,1.000,143.000,1.000,10000000001.000,10.000,1453190.000,3.000,2.000,...,18.000,507.000,2.000,2.000,1.000,1.000,101.000,-1.000,-1.000,-1.000
25%,2014.000,2.000,1.000,143.000,6.000,90000000362.750,4555.000,9202813790.750,12.000,2.000,...,39.000,9123157062.750,2.000,6.000,1.000,1.000,171.000,700000.000,4.000,194739514.000
50%,2014.000,2.000,1.000,143.000,7.000,160000000108.500,17800.000,30789791522.000,12.000,2.000,...,47.000,28014080398.000,2.000,7.000,3.000,1.000,277.000,2000000.000,5.000,305527014.000
75%,2014.000,2.000,1.000,143.000,7.000,210000000713.250,40999.000,59010387957.750,12.000,2.000,...,55.000,71325812828.250,4.000,8.000,3.000,3.000,581.000,3000000.000,5.000,656414514.000
max,2014.000,2.000,2.000,305.000,10.000,280000000123.000,90999.000,99986256020.000,12.000,17.000,...,92.000,416980390124.000,4.000,8.000,9.000,5.000,999.000,383000000.000,6.000,2209512014.000


# Eleições 2014

## Quantidade de candidatos?

In [25]:
candidatos_2014['NM_CANDIDATO'].count()

26264

## Quantidade de candidatos por gênero?

In [76]:
candidatos_2014.groupby(['DS_GENERO'])['NM_CANDIDATO'].count()

DS_GENERO
FEMININO      8139
MASCULINO    18125
Name: NM_CANDIDATO, dtype: int64

## Qual o grau de instrução dos candidatos por gênero?

In [60]:
temp = candidatos_2014.groupby(['DS_GENERO', 'DS_GRAU_INSTRUCAO'])['NM_CANDIDATO'].count().reset_index(name='DS_GENERO_GRAU_INSTRUCAO')
temp

,DS_GENERO,DS_GRAU_INSTRUCAO,DS_GENERO_GRAU_INSTRUCAO
0,FEMININO,ENSINO FUNDAMENTAL COMPLETO,557
1,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,271
2,FEMININO,ENSINO MÉDIO COMPLETO,2773
3,FEMININO,ENSINO MÉDIO INCOMPLETO,296
4,FEMININO,LÊ E ESCREVE,112
5,FEMININO,SUPERIOR COMPLETO,3286
6,FEMININO,SUPERIOR INCOMPLETO,844
7,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,1346
8,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,615
9,MASCULINO,ENSINO MÉDIO COMPLETO,5098


In [81]:
temp[temp['DS_GENERO'] == 'FEMININO'].sort_values(ascending=False, by='DS_GENERO_GRAU_INSTRUCAO' )

,DS_GENERO,DS_GRAU_INSTRUCAO,DS_GENERO_GRAU_INSTRUCAO
5,FEMININO,SUPERIOR COMPLETO,3286
2,FEMININO,ENSINO MÉDIO COMPLETO,2773
6,FEMININO,SUPERIOR INCOMPLETO,844
0,FEMININO,ENSINO FUNDAMENTAL COMPLETO,557
3,FEMININO,ENSINO MÉDIO INCOMPLETO,296
1,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,271
4,FEMININO,LÊ E ESCREVE,112


In [82]:
temp[temp['DS_GENERO'] == 'MASCULINO'].sort_values(ascending=False, by='DS_GENERO_GRAU_INSTRUCAO' )

,DS_GENERO,DS_GRAU_INSTRUCAO,DS_GENERO_GRAU_INSTRUCAO
12,MASCULINO,SUPERIOR COMPLETO,8645
9,MASCULINO,ENSINO MÉDIO COMPLETO,5098
13,MASCULINO,SUPERIOR INCOMPLETO,1729
7,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,1346
8,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,615
10,MASCULINO,ENSINO MÉDIO INCOMPLETO,542
11,MASCULINO,LÊ E ESCREVE,150


# DESAFIO:
# Prever a quantidade de eleitos/outros. (DS_SIT_TOT_TURNO)

In [117]:
candidatos_2014[['NR_DESPESA_MAX_CAMPANHA', 'CD_GENERO', 'CD_GRAU_INSTRUCAO', 'NR_IDADE_DATA_POSSE', 'CD_ESTADO_CIVIL', 'CD_COR_RACA']].describe()

,NR_DESPESA_MAX_CAMPANHA,CD_GENERO,CD_GRAU_INSTRUCAO,NR_IDADE_DATA_POSSE,CD_ESTADO_CIVIL,CD_COR_RACA
count,2.626400e+04,26264.000000,26264.000000,26264.000000,26264.000000,26264.000000
mean,2.492303e+06,2.619784,6.688585,47.171147,3.118794,1.819068
std,5.239805e+06,0.924915,1.491959,11.429958,2.300481,0.956349
min,-1.000000e+00,2.000000,2.000000,18.000000,1.000000,1.000000
25%,7.000000e+05,2.000000,6.000000,39.000000,1.000000,1.000000
50%,2.000000e+06,2.000000,7.000000,47.000000,3.000000,1.000000
75%,3.000000e+06,4.000000,8.000000,55.000000,3.000000,3.000000
max,3.830000e+08,4.000000,8.000000,92.000000,9.000000,5.000000


## Variaveis do KNN (X e y)

Escolha das variáveis:
* ST_REELEICAO (Indica se o candidato esta tentando uma reeleição ou não)
* NR_DESPESA_MAX_CAMPANHA ( Valor gasto na campanha)
* DS_GENERO (Gênero: Feminino ou Masculino)
* DS_GRAU_INSTRUCAO (Qual o grau de instrução do candidato)
* NR_IDADE_DATA_POSSE (Idade do candidato)
* DS_ESTADO_CIVIL (Diz o estado civil do candidato)
* DS_COR_RACA (Diz qual a raça do candidato)

Target:
* DS_SIT_TOT_TURNO (Descrição da situação de totalização do candidato naquele turno)

In [122]:
candidatos_2014[['ST_REELEICAO', 'NR_DESPESA_MAX_CAMPANHA', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 'NR_IDADE_DATA_POSSE', 'DS_ESTADO_CIVIL', 'DS_COR_RACA', 'DS_SIT_TOT_TURNO']]

,ST_REELEICAO,NR_DESPESA_MAX_CAMPANHA,DS_GENERO,DS_GRAU_INSTRUCAO,NR_IDADE_DATA_POSSE,DS_ESTADO_CIVIL,DS_COR_RACA,DS_SIT_TOT_TURNO
0,N,36000000,FEMININO,SUPERIOR COMPLETO,49,DIVORCIADO(A),BRANCA,NÃO ELEITO
1,N,20000000,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,50,CASADO(A),PARDA,NÃO ELEITO
2,N,2000000,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,46,DIVORCIADO(A),BRANCA,SUPLENTE
3,N,5000000,FEMININO,SUPERIOR COMPLETO,74,SOLTEIRO(A),BRANCA,SUPLENTE
4,N,1000000,FEMININO,SUPERIOR COMPLETO,48,CASADO(A),BRANCA,SUPLENTE
...,...,...,...,...,...,...,...,...
26259,N,3000000,MASCULINO,SUPERIOR COMPLETO,52,SOLTEIRO(A),PARDA,SUPLENTE
26260,N,4500000,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,59,CASADO(A),BRANCA,SUPLENTE
26261,N,3000000,FEMININO,SUPERIOR INCOMPLETO,41,CASADO(A),BRANCA,SUPLENTE
26262,N,1200000,FEMININO,SUPERIOR COMPLETO,27,SOLTEIRO(A),BRANCA,SUPLENTE


## Variável resultado (y_chapeu)

In [357]:
candidatos_2014.groupby(['DS_SIT_TOT_TURNO'])['DS_SIT_TOT_TURNO'].count()

DS_SIT_TOT_TURNO
#NULO#               3280
2º TURNO               60
ELEITO                143
ELEITO POR MÉDIA      215
ELEITO POR QP        1357
NÃO ELEITO           4992
SUPLENTE            16217
Name: DS_SIT_TOT_TURNO, dtype: int64

## Escolha do número de vizinhos. Total dividido pelo número de categorias.

In [349]:
candidatos_2014['DS_SIT_TOT_TURNO'].nunique()

7

In [381]:
candidatos_2014['DS_SIT_TOT_TURNO'].count()

26264

In [382]:
candidatos_2014['DS_SIT_TOT_TURNO'].count() / candidatos_2014['DS_SIT_TOT_TURNO'].nunique()

3752.0

## Tiramos a categoria 'SUPLENTE' por ter um número muito alto em relação aos demais.

In [380]:
candidatos_2014[candidatos_2014['DS_SIT_TOT_TURNO'] != 'SUPLENTE']['DS_SIT_TOT_TURNO'].nunique()

6

In [378]:
candidatos_2014[candidatos_2014['DS_SIT_TOT_TURNO'] != 'SUPLENTE']['DS_SIT_TOT_TURNO'].count()

10047

In [383]:
candidatos_2014[candidatos_2014['DS_SIT_TOT_TURNO'] != 'SUPLENTE']['DS_SIT_TOT_TURNO'].count() / candidatos_2014[candidatos_2014['DS_SIT_TOT_TURNO'] != 'SUPLENTE']['DS_SIT_TOT_TURNO'].nunique()

1674.5

## Usando KNN com 2 variavéis. ('ST_REELEICAO' e 'NR_DESPESA_MAX_CAMPANHA')

In [577]:
X = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')][['ST_REELEICAO','NR_DESPESA_MAX_CAMPANHA']]

## Transformando a coluna 'ST_REELEICAO' para 'int'

In [578]:
X['ST_REELEICAO'] = X['ST_REELEICAO'].cat.rename_categories([0,1])

In [579]:
y = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')]['DS_SIT_TOT_TURNO']

## Dividindo data set em 'train' e 'test'

In [580]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [641]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors= 1674, weights='distance')

# Train the model using the training sets
model.fit(X_train, np.array(y_train))

#Predict Output
predicted= model.predict(X_test) # 0:Overcast, 2:Mild
print(predicted)

[5 5 5 ... 5 5 5]


In [393]:
y_test

2650          SUPLENTE
6056          SUPLENTE
1249          SUPLENTE
15987         SUPLENTE
20772    ELEITO POR QP
             ...      
5411          SUPLENTE
12592         SUPLENTE
8940            #NULO#
3202          SUPLENTE
5261     ELEITO POR QP
Name: DS_SIT_TOT_TURNO, Length: 8658, dtype: category
Categories (7, object): [#NULO# < 2º TURNO < ELEITO < ELEITO POR MÉDIA < ELEITO POR QP < NÃO ELEITO < SUPLENTE]

In [385]:
novo_y_test = np.array(y_test)
novo_y_test

array(['SUPLENTE', 'SUPLENTE', 'SUPLENTE', ..., '#NULO#', 'SUPLENTE',
       'ELEITO POR QP'], dtype=object)

In [386]:
predicted

array(['SUPLENTE', 'SUPLENTE', 'SUPLENTE', ..., '#NULO#', 'SUPLENTE',
       'SUPLENTE'], dtype=object)

# Taxa de acerto (KNN)

In [387]:
1 - sum([y != p for y, p in zip(novo_y_test, predicted)]) /len(predicted)

0.782975282975283

# Usando Navi Bayes com 2 variáveis.

In [368]:
X = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')][['ST_REELEICAO','NR_DESPESA_MAX_CAMPANHA']]

In [369]:
y = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')]['DS_SIT_TOT_TURNO']

In [370]:
X['ST_REELEICAO'] = X['ST_REELEICAO'].cat.rename_categories([0,1])

## Dividindo data set em 'train' e 'test'

In [371]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [341]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [342]:
y_pred

array(['SUPLENTE', 'SUPLENTE', 'SUPLENTE', ..., '#NULO#', 'SUPLENTE',
       'SUPLENTE'], dtype='<U16')

In [344]:
novo_y_test = np.array(y_test)
novo_y_test

array(['SUPLENTE', 'SUPLENTE', 'SUPLENTE', ..., '#NULO#', 'SUPLENTE',
       'ELEITO POR QP'], dtype=object)

# Taxa de acerto (Navi Bayes)

In [345]:
1 - sum([y != p for y, p in zip(novo_y_test, y_pred)]) /len(y_pred)

0.7428967428967429

# Usando Random Forest e descobrindo variáveis importantes para o resultado

In [392]:
import pandas as pd
import numpy as np
from fastaai_utils import add_datepart, train_cats, proc_df
import os
import math
from sklearn.ensemble import RandomForestRegressor

# Escolha das variáveis do modelo Random Forest

In [604]:
modelo_tratamento = candidatos_2014[['NR_TURNO', 'TP_ABRANGENCIA','NM_UE','CD_CARGO','TP_AGREMIACAO', 'NR_PARTIDO', 'NR_IDADE_DATA_POSSE', 'CD_GENERO', 'CD_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL','CD_COR_RACA', 'NR_DESPESA_MAX_CAMPANHA', 'CD_SIT_TOT_TURNO','ST_REELEICAO']]

## Escolhendo todos os candidatos que tenham a despesa total de campanha maior do que zero

In [605]:
modelo_random = modelo_tratamento[modelo_tratamento['NR_DESPESA_MAX_CAMPANHA'] > 0]

In [606]:
import pandas as pd
import numpy as np
from fastaai_utils import add_datepart, train_cats, proc_df
import os
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [607]:
df_raw = modelo_random

In [611]:
df_raw.head().T

,0,1,2,3,4
NR_TURNO,1,1,1,1,1
TP_ABRANGENCIA,FEDERAL,FEDERAL,FEDERAL,FEDERAL,FEDERAL
NM_UE,PARANÁ,ALAGOAS,RIO DE JANEIRO,RIO DE JANEIRO,RIO DE JANEIRO
CD_CARGO,3,7,7,7,7
TP_AGREMIACAO,COLIGAÇÃO,PARTIDO ISOLADO,COLIGAÇÃO,PARTIDO ISOLADO,COLIGAÇÃO
NR_PARTIDO,13,36,36,31,25
NR_IDADE_DATA_POSSE,49,50,46,74,48
CD_GENERO,4,2,2,4,4
CD_GRAU_INSTRUCAO,8,4,3,8,8
CD_ESTADO_CIVIL,9,3,9,1,3


## Log do campo de depesa de campanha para normalizar os valores

In [612]:
df_raw.NR_DESPESA_MAX_CAMPANHA = np.log(df_raw.NR_DESPESA_MAX_CAMPANHA)

## Passando para 'int' as colunas com dtype 'category'

In [613]:
df_raw.ST_REELEICAO = df_raw.ST_REELEICAO.cat.codes

In [614]:
df_raw.TP_ABRANGENCIA = df_raw.TP_ABRANGENCIA.cat.codes

In [615]:
df_raw.NM_UE = df_raw.NM_UE.cat.codes

In [616]:
df_raw.TP_AGREMIACAO = df_raw.TP_AGREMIACAO.cat.codes

## Checar relação entre as variáveis

In [617]:
modelo = df_raw

In [618]:
modelo.corr()

,NR_TURNO,TP_ABRANGENCIA,NM_UE,CD_CARGO,TP_AGREMIACAO,NR_PARTIDO,NR_IDADE_DATA_POSSE,CD_GENERO,CD_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,CD_COR_RACA,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,ST_REELEICAO
NR_TURNO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
TP_ABRANGENCIA,nan,1.000,0.031,0.116,-0.001,0.010,-0.004,0.003,-0.016,-0.001,-0.011,-0.042,0.004,0.004
NM_UE,nan,0.031,1.000,-0.129,0.229,-0.005,0.050,-0.005,0.000,0.031,-0.214,0.203,-0.000,0.004
CD_CARGO,nan,0.116,-0.129,1.000,0.020,0.025,-0.065,0.022,-0.086,-0.016,0.062,-0.169,0.147,-0.023
TP_AGREMIACAO,nan,-0.001,0.229,0.020,1.000,0.018,0.005,0.009,0.000,0.011,-0.060,-0.065,-0.061,-0.039
NR_PARTIDO,nan,0.010,-0.005,0.025,0.018,1.000,-0.057,-0.013,-0.040,-0.012,0.043,-0.054,-0.007,-0.029
NR_IDADE_DATA_POSSE,nan,-0.004,0.050,-0.065,0.005,-0.057,1.000,-0.090,-0.013,0.269,-0.060,0.129,-0.050,0.115
CD_GENERO,nan,0.003,-0.005,0.022,0.009,-0.013,-0.090,1.000,-0.036,-0.019,0.019,-0.025,0.100,-0.075
CD_GRAU_INSTRUCAO,nan,-0.016,0.000,-0.086,0.000,-0.040,-0.013,-0.036,1.000,0.060,-0.137,0.047,-0.115,0.110
CD_ESTADO_CIVIL,nan,-0.001,0.031,-0.016,0.011,-0.012,0.269,-0.019,0.060,1.000,-0.051,0.071,-0.010,0.031


In [619]:
modelo.head().T

,0,1,2,3,4
NR_TURNO,1.000,1.000,1.000,1.000,1.000
TP_ABRANGENCIA,1.000,1.000,1.000,1.000,1.000
NM_UE,14.000,1.000,19.000,19.000,19.000
CD_CARGO,3.000,7.000,7.000,7.000,7.000
TP_AGREMIACAO,0.000,1.000,0.000,1.000,0.000
NR_PARTIDO,13.000,36.000,36.000,31.000,25.000
NR_IDADE_DATA_POSSE,49.000,50.000,46.000,74.000,48.000
CD_GENERO,4.000,2.000,2.000,4.000,4.000
CD_GRAU_INSTRUCAO,8.000,4.000,3.000,8.000,8.000
CD_ESTADO_CIVIL,9.000,3.000,9.000,1.000,3.000


# Modelo Random Forest Classifier

In [621]:
X = modelo.drop(['CD_SIT_TOT_TURNO'], axis=1)

In [622]:
y = modelo.CD_SIT_TOT_TURNO

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=84)

In [623]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

In [624]:
clf.fit(X_train, np.array(y_train))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

# Validando as variáveis mais influentes no resultado

In [625]:
print(clf.feature_importances_)

[0.         0.         0.01675173 0.12908351 0.18784238 0.07001589
 0.00470821 0.00867308 0.02015574 0.0024166  0.0015932  0.31917233
 0.23958733]


In [626]:
predicted = clf.predict(X_test)

## Teste com random_state de 84

In [627]:
acerto_com_random_state_84 = 1 - (sum([y != y_chapeu for y_chapeu, y in zip(predicted, y_test.values)]) / len(predicted))

In [628]:
acerto_com_random_state_84

0.7494913874949138

In [574]:
acerto

0.7542384375423844

# Rodando KNN com as variáveis com maior peso após o Random Forest Classifier

In [644]:
X = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')][['NR_DESPESA_MAX_CAMPANHA','ST_REELEICAO', 'CD_CARGO', 'TP_AGREMIACAO']]

In [645]:
X['ST_REELEICAO'] = X['ST_REELEICAO'].cat.rename_categories([0,1])

In [646]:
X.TP_AGREMIACAO = X.TP_AGREMIACAO.cat.codes

In [647]:
y = candidatos_2014[(candidatos_2014['DS_CARGO'] != 'PRESIDENTE')&(candidatos_2014['DS_CARGO'] != 'VICE-PRESIDENTE')]['DS_SIT_TOT_TURNO']

In [648]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

In [649]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors= 1674, weights='distance')

# Train the model using the training sets
model.fit(X_train, np.array(y_train))

#Predict Output
predicted= model.predict(X_test) # 0:Overcast, 2:Mild
print(predicted)

['#NULO#' 'NÃO ELEITO' 'NÃO ELEITO' ... 'SUPLENTE' 'SUPLENTE'
 'ELEITO POR QP']


In [650]:
novo_y_test = np.array(y_test)
novo_y_test

array(['#NULO#', 'NÃO ELEITO', 'NÃO ELEITO', ..., 'SUPLENTE', 'SUPLENTE',
       'ELEITO POR QP'], dtype=object)

In [651]:
predicted

array(['#NULO#', 'NÃO ELEITO', 'NÃO ELEITO', ..., 'SUPLENTE', 'SUPLENTE',
       'ELEITO POR QP'], dtype=object)

In [652]:
1 - sum([y != p for y, p in zip(novo_y_test, predicted)]) /len(predicted)

0.8304458304458304

In [654]:
resultado = pd.DataFramepredicted

AttributeError: type object 'DataFrame' has no attribute 'predicted'